18. Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo. (⭐⭐)


In [1]:
!pip install pyspark
!apt update

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=86c3eb1638518ed21d8748fab51ebc92d5f50b5cf1b6de7b391d21c760169d27
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,081 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.t

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [5]:
sqlContext = SQLContext(sc)
# Atencion, utilizo estos metodos para separar bien la forma en la que se lee del csv, ya que sino se lee de manera incorrecta
app_data = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('/content/drive/MyDrive/aa-OrgaDeDatos/googleplaystore.csv', header=True, inferSchema=True)
app_reviews = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('/content/drive/MyDrive/aa-OrgaDeDatos/googleplaystore_user_reviews.csv', header=True, inferSchema=True)
rdd_app_data = app_data.rdd
rdd_app_reviews = app_reviews.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
# Hago esto para poder "droppear" las repetidas. Use este metodo porque distinct elimina la fila si y solo si todos los campos de dos filas coinciden, cosa que
# no pasa en este caso. Hay apps que tienen todos los campos iguales y difieren por algunas reviews, lo que puede afectar al resultado final en este caso.
def get_key(x):
    return "{0}".format(x["App"])
rdd_app_data_no_duplicates= rdd_app_data.map(lambda x: (get_key(x), x)).reduceByKey(lambda x,y: x)

Esto lo hago para ver que hay en esta row, previo al collect hice un .count() para ver que se trataba de datos aceptables para usar collect

In [7]:
rdd_app_data_no_duplicates.map(lambda row: row[1]["Android Ver"]).distinct().collect()

['4.0.3 and up',
 '4.2 and up',
 '4.4 and up',
 '2.3 and up',
 '3.0 and up',
 '4.1 and up',
 '4.0 and up',
 '2.3.3 and up',
 'Varies with device',
 '2.2 and up',
 '5.0 and up',
 '6.0 and up',
 '1.6 and up',
 '1.5 and up',
 '2.1 and up',
 '7.0 and up',
 '5.1 and up',
 '4.3 and up',
 '4.0.3 - 7.1.1',
 '2.0 and up',
 '3.2 and up',
 '4.4W and up',
 '7.1 and up',
 '7.0 - 7.1.1',
 '8.0 and up',
 '5.0 - 8.0',
 '3.1 and up',
 '2.0.1 and up',
 '4.1 - 7.1.1',
 'NaN',
 '5.0 - 6.0',
 '1.0 and up',
 '2.2 - 7.1.1',
 '5.0 - 7.1.1',
 None]

**Aclaración:** Consulté por el canal de dudas de si estaba bien dejar las versiones asi o si en el caso de que dijera "and up" considerar todas las siguientes, y me aconsejaron que las dejara tal cual =). Por otra parte, la función utilizada para formatear el size ya la utilice para un ej de pandas y ya comprobe los datos que se utilizaban en esa columna.


Breve explicacion de lo que se hace a continuacion:

1. Filtro los varies with devices del size y los de la version de android junto a los NaN.


2. Transformo a clave-valor: (Android Ver, (tamanio en bytes, 1)) y posteriormente un reduceByKey para hacer la suma de los bytes y repeticiones por version de android.

3. Finalmente utilizo map para dividir cada valor total de bytes por su numero de repeticiones para obtener el promedio y hago collect() porque se que se trata de un conjunto acotado

El promedio se muestra en bytes


In [8]:
rdd_filtered_app_data = rdd_app_data_no_duplicates\
.filter(lambda row: row[1]["Android Ver"] is not None and row[1]["Android Ver"] != 'Varies with device' and row[1]["Android Ver"] != 'NaN' and row[1]["Size"] != 'Varies with device')\
.cache()

def formatear_espacio_a_bytes(tamanio):
    if('k' in tamanio):
        return float(tamanio.replace('k', '')) * 1024

    return float(tamanio.replace('M', '')) * 1024 * 1024


promedio_por_version = rdd_filtered_app_data.map(lambda row: (row[1]["Android Ver"], (formatear_espacio_a_bytes(row[1]["Size"]), 1)))\
.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1])).map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

promedio_por_version

[('4.0.3 and up', 22250844.868285507),
 ('4.2 and up', 22177551.229461752),
 ('4.4 and up', 26495785.93901808),
 ('2.3 and up', 21247357.21188121),
 ('3.0 and up', 14397302.580086576),
 ('4.1 and up', 25450903.456517633),
 ('4.0 and up', 17290505.980348628),
 ('2.2 and up', 8617520.596610168),
 ('5.0 and up', 24444366.166735094),
 ('6.0 and up', 20408175.435294118),
 ('1.6 and up', 3265945.6),
 ('2.1 and up', 5852814.436090228),
 ('5.1 and up', 25231018.666666668),
 ('1.5 and up', 5316101.389473684),
 ('7.0 and up', 25495214.080000006),
 ('4.3 and up', 22988220.377358496),
 ('4.0.3 - 7.1.1', 6920601.6),
 ('2.0 and up', 6557834.735483871),
 ('2.3.3 and up', 19568689.671641793),
 ('3.2 and up', 10339436.08888889),
 ('4.4W and up', 14813519.127272727),
 ('7.1 and up', 33558784.0),
 ('7.0 - 7.1.1', 7444889.6),
 ('8.0 and up', 21058901.333333332),
 ('5.0 - 8.0', 24956108.8),
 ('3.1 and up', 14064558.080000002),
 ('2.0.1 and up', 23299306.057142857),
 ('4.1 - 7.1.1', 8283750.4),
 ('5.0 - 6.0